In [94]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from fastcore.all import *
import pandas as pd
import json
from datetime import datetime
import re
import json
from urllib.request import urlopen

In [2]:
main_url = 'https://help.dyalog.com/18.2/Content/Language/Introduction/Language%20Elements.htm?tocpath=Language%20Reference%20Guide%7CSymbols%7C_____1'
content = requests.get(main_url)
content.encoding = content.apparent_encoding

main=BeautifulSoup(content.text)

symbol_tags = main.find_all('a',attrs={'href':lambda x: x.startswith('../Symbols/')})
symbols_dict = {o.text:o['href'] for o in symbol_tags}

# General Cards from Dyalog Docs

In [3]:
base = 'https://help.dyalog.com/18.2/Content/Language'

In [4]:
def get_hyperlink(url,url_type):
    return f" <br/><br/><a href='{url}'>Take me to {url_type}</a>"

In [95]:
card_names = []
cards = []
for i,sym in enumerate(L(*symbols_dict.keys())):
    doc_url = symbols_dict[sym]
    url1 = base + doc_url[2:]

    content = requests.get(url1)
    content.encoding = content.apparent_encoding
    doc_html = BeautifulSoup(content.text)

    doc_h5s = doc_html.find_all('h5')
    doc_h3 = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
    
    
    if doc_html.find('td').text.strip() == '':
        _2 = doc_h3['name']
    else: _2 = doc_html.find('td').text.strip()
    
    # Name of glyph -> Symbol
    card_names.append(pd.DataFrame([[_2 + "<br/><br/>What does this glyph look like?",
                                sym + get_hyperlink(url1,"symbol documentation"),
                               "Name->Sym"
                               ]]))
    urls = L(url1)
    docs = ''
    for doc_h5 in doc_h5s:
        
        p = doc_h5.find_next_sibling('p')
        dy_or_mon_name = ''.join(re.findall('Dyadic|Monadic',doc_h5.text)) + ': ' + p.text.strip()
        
        # Mondadic/Dyadic Name -> Symbol
        card_names.append(pd.DataFrame([[dy_or_mon_name + "<br/><br/>What does this glyph look like?",
                                    sym + get_hyperlink(url1,"symbol documentation"),
                                   "Name->Sym"
                                   ]]))
        _2  = _2 + ' <br/> ' + dy_or_mon_name 
        
        
        url_end = [o.find('a',attrs={'href':True}) for o in doc_h5.find_next_siblings('p') if o.find('a',attrs={'href':True}) is not None][0]['href']
#         url2 = base + p.find('a',attrs={'href':True})['href'][2:]
        url2 = base + url_end[2:]
        urls.append(url2)
        
        content = requests.get(url2)
        content.encoding = content.apparent_encoding
        doc_html = BeautifulSoup(content.text)
    
        try:
            desc = [o.text for o in doc_html.find('td').find_all_next('p') if o.text.strip() != '']

            documentation = '<br/>'.join(desc)
            docs = docs + documentation + '<br/><br/>' 
            # Functional documentation -> Symbol
            cards.append(pd.DataFrame([[documentation + "<br/><br/>What does this glyph look like?",
                                        sym + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                                       "Docs->Sym"
                                       ]]))
        except:
            print(i)
        try:
            doc_name = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
            if doc_name['name'] not in _2:
                # Documentation Name -> Symbol
                cards.append(pd.DataFrame([[doc_name['name'] + "<br/><br/>What does this glyph look like?",
                                            sym + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                                           "Name->Sym"
                                           ]]))

            # Documentation Name -> Functional Documentation
            # Functional Documentation -> Documentation Name
            cards.append(pd.DataFrame([
                    [doc_name['name'] + "<br/><br/>What does this glyph do?",
                     documentation + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                     "Name->Docs"
                    ],
                    [documentation + "<br/><br/>What is the name of this glyph?",
                     doc_name['name'] + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                    "Docs->Name"
                    ],
                               
                    ]))
        except:
            print(i)
    
    # Symbol -> Functional Documentation
    cards.append(pd.DataFrame([[sym + "<br/><br/>What does this glyph do?",
                                        docs + ''.join([get_hyperlink(o,"Link") for o in urls.unique()]),
                                       "Sym->Docs"
                                       ]]))
    # Symbol -> Names
    card_names.append(pd.DataFrame([[sym + "<br/><br/>What is this glyph called?  What's the Monadic/Dyadic names?",
                                _2 + get_hyperlink(url1,"symbol documentation"),
                               "Sym->Name"]]))

65
72
72
72


In [6]:
df = pd.concat(card_names)
df.to_csv('decks/APL_DyalogDocs_Names.csv',index=False,header=False)
df = pd.concat(cards)
df.to_csv('decks/APL_DyalogDocs_Functionality.csv',index=False,header=False)

### Stuff that doesn't work quite right
+ A few documentation pages have code cells mixed into the text.  Currently it only pulls \<p> text cells so the code is lost.  That could be fixed, but it's infrequent and not a huge deal since there's hyperlinks.
+ Fix the try/except stuff

# Cards from study group NB

In [106]:
import requests
from IPython.core.display import HTML
from bs4 import BeautifulSoup

from fastcore.all import *
import pandas as pd
import json
from datetime import datetime
import json
from urllib.request import urlopen

In [107]:
nburls = ['https://raw.githubusercontent.com/fastai/apl-study/master/01_APL.ipynb',
        'https://raw.githubusercontent.com/fastai/apl-study/master/02_functions.ipynb',
       'https://raw.githubusercontent.com/fastai/apl-study/master/03_operators.ipynb',
        'https://raw.githubusercontent.com/fastai/apl-study/master/04_competition.ipynb',
        'https://raw.githubusercontent.com/fastai/apl-study/master/05_the_rest.ipynb']

base = 'https://help.dyalog.com/18.2/Content/Language'

In [108]:
# Define newlines on cards
def collapse_list(l): return '<br/>'.join(l) + '<br/>'

def get_hyperlink(url,url_type):
    return f" <br/><br/><a href='{url}'>Take me to {url_type}</a>"

In [109]:
main_url = 'https://help.dyalog.com/18.2/Content/Language/Introduction/Language%20Elements.htm?tocpath=Language%20Reference%20Guide%7CSymbols%7C_____1'
content = requests.get(main_url)
content.encoding = content.apparent_encoding

main=BeautifulSoup(content.text)
symbol_tags = main.find_all('a',attrs={'href':lambda x: x.startswith('../Symbols/')})
symbols_dict = {o.text:o['href'] for o in symbol_tags}

### Code Cells

In [110]:
def cell_filter(cell):
    cond1 = cell['cell_type'] == 'code'
    cond2 = ('outputs' in cell.keys()) and (cell['outputs'] != [])
    cond3 = cell['source'] != []
    return cond1 and cond2 and cond3

In [111]:
code_cards = []
error_cards = []
base = 'https://help.dyalog.com/18.2/Content/Language'

for nburl in nburls:
    nb = json.loads(urlopen(nburl).read())
    filtered_codes = L(nb['cells']).filter(cell_filter)

    for i,cell in enumerate(filtered_codes):
        _src = cell['source']

        _split = [[char for char in line] for line in _src]

        _split = [o for sublist in _split for o in sublist]
        _symbol_locs = {o:symbols_dict[o] for o in _split if o in filter_dict(symbols_dict,lambda k,v: k!='.').keys()}

        src = collapse_list(_src)
        
        try:
            out = ''.join(cell['outputs'][0]['data']['text/html'])
            out = out + ''.join([get_hyperlink(base+u[2:],s) for s,u in _symbol_locs.items()])
            code_cards.append(pd.DataFrame([[src + f" <br/><br/>What does the code output?",out,"Code->Output"]]))
        except:
            error_cards.append(cell)


In [112]:
print(len(code_cards),len(error_cards))
df = pd.concat(code_cards)
df.to_csv('decks/APL_FastaiStudyGroup_CodeCards.csv',index=False,header=False)

439 1


### Header Cells

In [116]:
def cell_filter(cell):
    cond1 = cell['cell_type'] == 'markdown'
    cond2 = ('source' in cell.keys()) and (cell['source'] != [])
    cond3 = (cell['source'] != []) and (cell['source'][0].startswith('##'))
    return cond1 and cond2 and cond3

In [119]:
code_cards = []
error_cards = []
base = 'https://help.dyalog.com/18.2/Content/Language'
for nburl in nburls:
    nb = json.loads(urlopen(nburl).read())
    filtered_codes = L(nb['cells']).filter(cell_filter)

    for i,cell in enumerate(filtered_codes):        
        try:
            symbol = re.findall("(?<=`).(?=`)",cell['source'][0])
            cell_type =  re.findall("(?<=# ).+(?=`.`)",cell['source'][0])
            name =  re.findall("(?<=`.`).+",cell['source'][0])
            assert len(symbol)==1 and len(cell_type)==1 and len(name) == 1

            symbol,cell_type,name = symbol[0],cell_type[0],name[0]
            docs_link = get_hyperlink(base + symbols_dict[symbol[0]][2:],symbol[0])
            
            if cell_type == []:
                front = symbol + "<br/><br/>What's the name?"
                back = name + docs_link
                code_cards.append(pd.DataFrame([[front,back]]))
                front = name + "<br/><br/>What's the symbol look like?"
                back = symbol + docs_link
                code_cards.append(pd.DataFrame([[front,back]]))
            else:
                front = cell_type + ' ' + symbol + "<br/><br/>What's the name?"
                back = name + docs_link
                code_cards.append(pd.DataFrame([[front,back]]))
                
                front = cell_type + '<br/><br/>' + name + "<br/><br/>What's the symbol look like?"
                back = symbol + docs_link
                code_cards.append(pd.DataFrame([[front,back]]))
            
        except:
            error_cards.append(cell)   

In [120]:
df = pd.concat(code_cards)
df.to_csv('decks/APL_FastaiStudyGroup_MdCards.csv',index=False,header=False)

# APL Cart

In [14]:
url = 'https://raw.githubusercontent.com/abrudz/aplcart/master/table.tsv'
df = pd.read_csv(url, sep='\t')

In [15]:
front = df['SYNTAX']
back = df['DESCRIPTION'].astype(str) + ' <br/>'*2 + "<a href='" + df['DOCS'].astype(str) + "'>Link to docs<a/>"
g1 = pd.DataFrame({'front':front,'back':back})

In [16]:
front = df['DESCRIPTION']
back = df['SYNTAX'].astype(str) + ' <br/>'*2 + "<a href='" + df['DOCS'].astype(str) + "'>Link to docs<a/>"
g2 = pd.DataFrame({'front':front,'back':back})

In [17]:
cards = pd.concat([g1,g2])
cards.to_csv('decks/APL_AplCart.csv',index=False,header=False)